# Trendvorhersage Topics

Bisher hast du nur *extensive Größen* vorhergesagt. Es könnte sein, dass das Wachstum des Flairs nur mit dem Wachstum von Reddit alleine zusammenhängt.

*Intensive Größen* haben diese Probleme nicht. Eine solche kennst du schon, nämliche die Verteilung der Topics. Dazu wendest du in diesem Notebook nochmal das Topic Model an.

## Nutzung für die Reddit-Daten

Lade auch hier zuerst wie gewohnt die Daten in einen `DataFrame`.

In [1]:
import pandas as pd

posts = pd.read_csv("transport-all-comments.csv.xz", parse_dates=["created_utc"])

## Topic-Größen vorhersagen

Den Code kennst du schon von den Topic Models.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
for w in "removed deleted post message account moderators http https www youtube com \
          watch gt look looks feel test know think go going submission link apologize \
          inconvenience don want automatically based buy compose good image karma like \
          lot need people self shit sound sounds spam submitting subreddit things \
          video way years time days doesn en fuck money org read reddit review \
          right said says subreddit subreddits sure thank try use videos wiki \
          wikipedia work ll thing point ve actually wait hello new amp better \
          isn yeah probably pretty yes didn pay long posts commenting portion \
          contribute questions unfortunately allowed submissions gifs pics sidebar".split(" "):
    stopwords.add(w)

ModuleNotFoundError: No module named 'spacy'

### Vektorisierung

In [ ]:
tfidf_text_vectorizer = TfidfVectorizer(stop_words=stopwords, min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(posts['text'])

### Topic-Modelle berechnen

In [ ]:
from sklearn.decomposition import NMF
nmf_text_model = NMF(n_components=10, random_state=42)
W_text_matrix = nmf_text_model.fit_transform(tfidf_text_vectors)

Du verwendest die vereinfachte Darstellung der Topics als Texte:

In [ ]:
def display_topics(model, features, no_top_words=5):
    for topic, words in enumerate(model.components_):
        total = words.sum()
        largest = words.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]], abs(words[largest[i]]*100.0/total)))

In [ ]:
display_topics(nmf_text_model, tfidf_text_vectorizer.get_feature_names_out())

Du möchtest eine Vorhersage auf Monatsbasis durchführen. Dazu musst du das Datum der einzelnen Posts auf Monate runden. Die `resample`-Funktion kannst du hier leider nicht nutzen, weil damit auch immer eine Aggregation verbunden ist, die dir hier im Wege stehen würde:

In [ ]:
posts["month"] = posts.index.strftime("%Y-%m")

Anschließend verwendest du eine Schleife, um über alle Monate zu iterieren. Du wendest das Topic Model auf die Textdaten des jeweiligen Monats an - dabei hilft dir `numpy` mit komfortablen Selektionsmöglichkeiten der TF/IDF-Vektoren.

In [ ]:
import numpy as np
month_data = []
for month in np.unique(np.unique(posts["month"])):
    W_month = nmf_text_model.transform(tfidf_text_vectors[np.array(posts["month"] == month)])
    month_data.append([month] + list(W_month.sum(axis=0)/W_month.sum()*100.0))

Gib den Topics nun kurze Namen, die aus den zwei wichtigsten Wörtern bestehen.

In [ ]:
topic_names = []
voc = tfidf_text_vectorizer.get_feature_names_out()
for topic in nmf_text_model.components_:
    important = topic.argsort()
    top_word = voc[important[-1]] + " " + voc[important[-2]]
    topic_names.append("Topic " + top_word)

Die Ergebnisse kannst du gut in einem Area-Plot darstellen:

In [ ]:
df_month = pd.DataFrame(month_data, columns=["month"] + topic_names).set_index("month")
df_month.plot.area(figsize=(16, 9))

### Trendvorhersage für Topics

Den Anteil des `tesla model`-Topics kannst du nun als Größe nutzen, für die du den Trend vorhersagen möchtest:

In [ ]:
from prophet import Prophet
df = pd.DataFrame({"ds": df_month.index.values, 
                   "y": df_month["Topic tesla model"].values})

m = Prophet()
m.fit(df)

Das führst du wieder für zwei Monate in die Zukunft aus:

In [ ]:
future = m.make_future_dataframe(periods=24, freq='M')
forecast = m.predict(future)
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

Das Tesla-Topic scheint also weiter an Gewicht zu gewinnen. Es bleibt abzuwarten, ob mehr und mehr Hersteller von Elektrofahrzeugen dort nicht noch ihren Einfluss einbringen können